# Análisis exploratorio de las publicaciones con descuento
Nota: Las publicaciones con precio con descuento son aquellas que poseen el campo original_price
!= null

In [22]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
from tqdm.notebook import tqdm

In [90]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None


# 1️⃣ Dataload 
---

In [34]:
url_base = "https://api.mercadolibre.com/"

In [32]:
url_MLA = url_base + "/sites/MLA/"

In [4]:
limit = 50 # fixed

In [5]:
items_per_category = 1000 # chosen

In [14]:
requests_per_category = int(items_per_category / limit)
requests_per_category

20

## Fxs

In [6]:
def get_data(url):
    # makes a get request given a url, and returns json response
    response = urllib.request.urlopen(url)
    return json.load(response)
    

In [16]:
def load_category_items(categ_id):
    # gets items given a category id
    url_categ = url_MLA + "search?&category={}".format(categ_id)
    all_results = [] # offset and limit must be taken into account to reach items_per_category
    for r in range(0, requests_per_category):
        offset = r * limit
        url = url_categ + "&offset={}".format(offset)
        response = get_data(url)
        results = response['results'] # keep only the items
        all_results = all_results + results
    return all_results 
    

Test

all_results = load_category_items("MLA1367")


len(all_results)

## Get root categories

In [11]:
url = url_MLA + "categories"

In [48]:
root_categories = get_data(url)
root_categories

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas y Construcción'},
 {'id': 'MLA1574', 'name': 'Hogar, Muebles y Jardín'},
 {'

## Get items per root category

In [24]:
df = pd.DataFrame() # will contain items from all root categories
for c in tqdm(root_categories):
    all_results = load_category_items(c["id"])
    df_tmp = pd.json_normalize(all_results) # build a pd df from a json with nested data (notice how lists remain lists)
    # add root category data to the df with category items
    df_tmp["root_category_id"] = c["id"]
    df_tmp["root_category_name"] = c["name"]
    # append df_tmp to the final df
    df = df.append(df_tmp, ignore_index=True)

  0%|          | 0/31 [00:00<?, ?it/s]

**Inspect final df**

In [25]:
df.shape

(30937, 132)

In [93]:
df.columns.tolist()

['id',
 'site_id',
 'title',
 'price',
 'sale_price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'buying_mode',
 'listing_type_id',
 'stop_time',
 'condition',
 'permalink',
 'thumbnail',
 'thumbnail_id',
 'accepts_mercadopago',
 'attributes',
 'original_price',
 'category_id',
 'official_store_id',
 'domain_id',
 'catalog_product_id',
 'tags',
 'catalog_listing',
 'use_thumbnail_id',
 'order_backend',
 'seller.id',
 'seller.permalink',
 'seller.registration_date',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.tags',
 'seller.eshop.nick_name',
 'seller.eshop.eshop_rubro',
 'seller.eshop.eshop_id',
 'seller.eshop.eshop_locations',
 'seller.eshop.site_id',
 'seller.eshop.eshop_logo_url',
 'seller.eshop.eshop_status_id',
 'seller.eshop.seller',
 'seller.eshop.eshop_experience',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transacti

Inspect specific columns values

In [ ]:
[x for x in df.columns if "address" in x]

In [ ]:
df['address.city_id'][0]

In [ ]:
df.category_id[0]

## Get subcategories info
Recursive algorithm

## TODO: metodo recursivo para ir pidiendo las subcategorias, asi le mapeo la categoria padre y la específica a cada item

- no se si sirve hacer una lista de todo el arbol de categorias????? -- emb?
- o bien mapear la categ específica a cada row ..... para aperturar el análisis que haga, y no quedarme con lo que veo a nivel root category nada más

In [69]:
def get_subcategories_from_root(categ_id, dic):
    # recursive function to get subcategories data 
    url = url_base + "categories/{}".format(categ_id)
    response = get_data(url)
    children_categories = response["children_categories"]
    for cc in children_categories: # base case: children_categories list is empty
        id = cc["id"]
        dic[id] = {"name": cc["name"], 
                    "root": response["path_from_root"]} # 'root' does not include current category as response["path_from_root"] has parent category data 
        get_subcategories_from_root(id, dic)  

Test 

In [70]:
dic = {}
res = get_subcategories_from_root("MLA4711", dic) # INFO: takes 2 minutes

In [71]:
dic

{'MLA86379': {'name': 'Alarmas para Motos',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'}]},
 'MLA434708': {'name': 'Alforjas',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'}]},
 'MLA11090': {'name': 'Baúles y Anclajes',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'}]},
 'MLA86350': {'name': 'Baúles Laterales',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'},
   {'id': 'MLA11090', 'name': 'Baúles y Anclajes'}]},
 'MLA432921': {'name': 'Baúles para Cuatriciclos',
  'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
   {'id': 'MLA4711', 'name': 'Acc. para Motos y Cuatriciclos'},
   {'id': 'MLA11090', 'name': 'Baúles y Anclajes'}]},
 'MLA379646': {'

**Get subcategories info for all root categories**

In [124]:
# FIXME: takes a lot of time ... maybe just get subcategies from selected group of root categories instead of all

In [72]:
dic_all_categories = {}
for c in tqdm(root_categories):
    dic = {}
    get_subcategories_from_root(c["id"], dic)  
    dic_all_categories.update(dic)

  0%|          | 0/31 [00:00<?, ?it/s]

{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'}
{'id': 'MLA1512', 'name': 'Agro'}
{'id': 'MLA1403', 'name': 'Alimentos y Bebidas'}
{'id': 'MLA1071', 'name': 'Animales y Mascotas'}
{'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'}
{'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'}
{'id': 'MLA1743', 'name': 'Autos, Motos y Otros'}
{'id': 'MLA1384', 'name': 'Bebés'}
{'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'}
{'id': 'MLA1039', 'name': 'Cámaras y Accesorios'}
{'id': 'MLA1051', 'name': 'Celulares y Teléfonos'}
{'id': 'MLA1648', 'name': 'Computación'}
{'id': 'MLA1144', 'name': 'Consolas y Videojuegos'}
{'id': 'MLA1276', 'name': 'Deportes y Fitness'}
{'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'}
{'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'}
{'id': 'MLA2547', 'name': 'Entradas para Eventos'}
{'id': 'MLA407134', 'name': 'Herramientas y Construcción'}
{'id': 'MLA1574', 'name': 'Hogar, Muebles y Jardín'}
{'id': 'MLA1499', 'name': 'Industrias y O

**Inspect dic_all_categories**

In [125]:
len(dic_all_categories)

9545

In [131]:
first_key = next(iter(dic_all_categories)) 
first_key

'MLA4711'

In [132]:
dic_all_categories[first_key]

{'name': 'Acc. para Motos y Cuatriciclos',
 'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'}]}

# 2️⃣ Manipulate df
---

In [74]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,seller.car_dealer_logo,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms
0,MLA833231588,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12800.0,None,ARS,15,1077,buy_it_now,gold_special,2040-08-01T20:21:12.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'id': 'BRAND', 'value_struct': None, 'value_...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[brand_verified, good_quality_picture, good_qu...",True,True,1,9696552,http://perfil.mercadolibre.com.ar/MUNDOJW.,2002-05-31

## Add columns

**Add discount column**

In [76]:
df["discount"] = 1 - df.price / df.original_price if df.original_price is not None else None

In [122]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,seller.car_dealer_logo,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount,category_name,category_full_name
0,MLA833231588,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12800.0,None,ARS,15,1077,buy_it_now,gold_special,2040-08-01T20:21:12.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'id': 'BRAND', 'value_struct': None, 'value_...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[brand_verified, good_quality_picture, good_qu...",True,True,1,9696552,http://perf

Inspect discount coulumn

In [86]:
df[~df.original_price.isna()][["id", "site_id", "title", "price", "sale_price", "original_price", "discount"]]

,id,site_id,title,price,sale_price,original_price,discount
7,MLA836983125,MLA,Funda Cubre Rueda Auxilio Ford Ecosport,891.00,None,990,0.1
14,MLA897397255,MLA,Camara Espejo Retrovisor Full Hd + Trasera Est...,2999.00,None,3999,0.250063
15,MLA785918507,MLA,Kit Led Cree Cob H1 H7 H3 9005 12/24v 2019 C6+...,999.00,None,1600,0.375625
17,MLA836882881,MLA,Mini Compresor De Aire Portatil 12v 250 Psi Kl...,1003.00,None,1249,0.196958
18,MLA836881964,MLA,Compresor De Aire Alto Impacto Portátil 250 Ps...,2127.00,None,2799,0.240086
...,...,...,...,...,...,...,...
30858,MLA905514102,MLA,Rack Mineria Rig Minero 6 Gpus Aluminio,7599.05,None,7999,0.05
30870,MLA879463123,MLA,Sahumo Sagrada Madre Limpieza Energética - Lla...,227.85,None,245,0.07
30887,MLA870837169,MLA,Turmalina Negra (chica) - Pacha Kuyuy,161.50,None,170,0.05
30890,MLA834986557,MLA,Resina De Jazmin India 50 Gr,218.50,None,230,0.05


**Add category name (specific one for each item)**

In [95]:
df["category_name"] = df.category_id.apply( lambda x: dic_all_categories[x]["name"] if x in dic_all_categories.keys() else None )

In [133]:
def build_str_from_root_info(lis):
    str = ""
    for r in lis[1:]: # avoid root (column root_category_name already has that info)
        str += r["name"] + " | "
    return str[:-2]

In [134]:
df["category_full_name"] = df.category_id.apply( lambda x: build_str_from_root_info(dic_all_categories[x]["root"]) if x in dic_all_categories.keys() else None )

In [135]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,seller.car_dealer_logo,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount,category_name,category_full_name
0,MLA833231588,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12800.0,None,ARS,15,1077,buy_it_now,gold_special,2040-08-01T20:21:12.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'id': 'BRAND', 'value_struct': None, 'value_...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[brand_verified, good_quality_picture, good_qu...",True,True,1,9696552,http://perf

## Filter out unwanted columns

In [ ]:
columns = [
    
]

In [140]:
df_final = df.copy()[[columns]] # drop by name: df1 = df1.drop(['B', 'C'], axis=1)
df_final.head()

SyntaxError: invalid syntax (<ipython-input-140-9c78185c7a7a>, line 1)